In [14]:
import pandas as pd
import os
from multiprocessing import Pool
import merge
import pickle
import time

In [17]:
def ispByCountry(filename):
    start_tm = time.time()
    labels=["IP From","IP To","Country Code","Country Name","Region Name","City Name","Latitude","Longitude",
                "Zipcode","Time Zone","ISP","Domain","Net speed","IDD Code","Area Code","Weather Station Code",
                "Weather Station Name","MCC","MNC","Mobile Brand","Elevation","Usage Type (Reserved)"]
    data = pd.read_csv("Data/"+filename,header=None,names=labels,skiprows=1, usecols=["IP From","IP To","Country Name","ISP"])
    
    countries = data.loc[:,"Country Name"]
    countries = countries.drop_duplicates()
    #country_dict = {key:[] for key in countries.tolist()}

    unique_counts = {}
    distrib_counts = {}
    for country in countries.tolist():
        df = data.loc[data["Country Name"] == country]
        isp_list = df.loc[0:,"ISP"].drop_duplicates().tolist()
        #country_dict[country] = isp_list
        unique_counts[country] = len(isp_list)
        
        distrib_counts[country] = {}
        for isp in isp_list:
            selected = df.loc[df["ISP"] == isp]
            
            ip_cnt = 0
            for start,end in zip(selected["IP From"],selected["IP To"]):
                ip_cnt = ip_cnt + end - start + 1 #+1 to make it inclusive
                
            distrib_counts[country][isp] = ip_cnt
    delta_tm = time.time() - start_tm
    m, s = divmod(delta_tm, 60)
    h, m = divmod(m, 60)
    print(filename + " finished counting IPs in %d:%02d:%02d" % (h, m, s))

    #return (unique_counts, distrib_counts)
    pickle.dump(distrib_counts,open("Results/"+filename.split(".")[0]+".p","wb"))
    return distrib_counts

In [18]:
#result = ispByCountry("sample.txt")
result = ispByCountry("data_1.csv")
#os.system('say "your program has finished"')
print("Done")

sample.txt finished counting IPs in 0:00:00
Done


In [ ]:
p = Pool()
parts = p.map(ispByCountry,['data_2.csv','data_3.csv','data_4.csv','data_5.csv','data_6.csv','data_1.csv','data_7.csv'])
print("Pool is done")

In [ ]:
p.close()
p.join()
result = merge.merge_dicts(parts + [test])
pickle.dump(result, open("result.p","wb"))
#result